# Train a Model

In [10]:
from pyxtal_ff import PyXtal_FF
import os
import urllib.request

In [7]:
TrainData = "training.json"
TestData  = "test.json"

if not os.path.exists(TrainData):
    if not os.path.exists('data'):
        os.mkdir('data')
    os.chdir('data')
    print('Downloading the training and test data')

    url = "https://raw.githubusercontent.com/materialsvirtuallab/mlearn/master/data/Si/"

    urllib.request.urlretrieve(f"{url}/{TrainData}", TrainData)
    urllib.request.urlretrieve(f"{url}/{TestData}", TestData)
    os.chdir('..')

In [8]:
if True:
    folder = 'Si-snap-zbl/'
    descriptor = {'type': 'SNAP',
                  'weights': {'Si': 1.0},
                  'Rc': 5.0,
                  'parameters': {'lmax': 3},
                  'base_potential': {'inner': 1.5, 'outer': 2.0}, #zbl potential
                  'ncpu': 1,
                 }
else:
    # Not working so far
    folder = 'Si-so3-zbl/'
    descriptor = {'type': 'SO3',
                  'weights': {'Si': 1.0},
                  'Rc': 5.0,
                  'parameters': {'lmax': 4, 'nmax': 3},
                  'base_potential': {'inner': 1.5, 'outer': 2.0}, #zbl potential
                  'ncpu': 1,
                 }

In [9]:
model = {'system' : ['Si'],
         'hiddenlayers': [12, 12],
         'path': folder,
         #'restart': folder + '12-12-checkpoint.pth',
         'optimizer': {'method': 'lbfgs'},
         'force_coefficient': 2e-2,
         'stress_coefficient': 2e-3,
         "stress_group": ["Elastic"],
         'alpha': 1e-6,
         'epoch': 1000,
         }

In [11]:
ff = PyXtal_FF(descriptors=descriptor, model=model)
ff.run(mode='train', TrainData=os.path.join("data", TrainData), TestData=os.path.join("data", TestData))




               ______       _    _          _         _______ _______ 
              (_____ \     \ \  / /        | |       (_______|_______)
               _____) )   _ \ \/ / |_  ____| |        _____   _____   
              |  ____/ | | | )  (|  _)/ _  | |       |  ___) |  ___)  
              | |    | |_| |/ /\ \ |_( ( | | |_______| |     | |      
              |_|     \__  /_/  \_\___)_||_|_(_______)_|     |_|      
                     (____/      


        A Python package for Machine Learning Interatomic Force Field
         Developed by Zhu's group at University of Nevada Las Vegas
    The source code is available at https://github.com/qzhu2017/FF-project


=========================== version 0.1.9 =============================

Descriptor parameters:
type        : SNAP
Rc          : 5.0
cutoff      : cosine
lmax        : 3
rfac        : 0.99363




UnboundLocalError: local variable 'structure_dict' referenced before assignment

# Run MD with ASE using this model

In [13]:
from time import time
from optparse import OptionParser
from ase.build import bulk
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.langevin import Langevin
from ase import units
from pyxtal_ff import PyXtal_FF
from pyxtal_ff.calculator import PyXtalFFCalculator

In [14]:
def printenergy(a, it, t0):
    """Function to print the potential, kinetic and total energy"""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    t_now = time()
    print('Step: %4d [%6.2f]: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV ' % (\
          it, t_now-t0, epot, ekin, ekin / (1.5 * units.kB), epot + ekin))
    return t_now

In [15]:
file = "Si-snap-zbl"

In [16]:
ff = PyXtal_FF(model={'system': ["Si"]}, logo=False)
ff.run(mode='predict', mliap=file)
calc = PyXtalFFCalculator(ff=ff)

si = bulk('Si', 'diamond', a=5.659, cubic=True)
si = si*5
print("MD simulation for ", len(si), " atoms")
si.set_calculator(calc)

MaxwellBoltzmannDistribution(si, 1000*units.kB)
dyn = Langevin(si, timestep=5 * units.fs, temperature_K=1000, friction=0.02)
#dyn = VelocityVerlet(si, 5*units.fs)  # 2 fs time step.
t0 = time()
for i in range(10):
    dyn.run(steps=1)
    t_now = printenergy(si, i, t0)
    t0 = t_now

PermissionError: [Errno 13] Permission denied: 'Si-snap-zbl'